In [309]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [310]:
import random

import fasttext
from tqdm import tqdm

In [311]:
def preprocess(text):
    # lower
    text = text.lower()

    # replace all non-alphanumeric characters with spaces
    chars = [c if c.isalnum() else " " for c in text]
    text = " ".join(
        # remove empty words
        [word for word in "".join(chars).split() if len(word) > 0]
    )

    return text

def txts_to_datset(prof_file, non_prof_file, dataset_file):
    prof_lines = []
    non_prof_lines = []

    with open(prof_file) as f:
        for line in tqdm(f):
            prof_lines.append((preprocess(line), "professional"))

    with open(non_prof_file) as f:
        for line in tqdm(f):
            non_prof_lines.append((preprocess(line), "not_professional"))

    random.Random(42).shuffle(prof_lines)
    random.Random(43).shuffle(non_prof_lines)

    limit = min(len(prof_lines), len(non_prof_lines))
    train_data = prof_lines[:limit] + non_prof_lines[:limit]
    random.Random(44).shuffle(train_data)

    with open(dataset_file, 'w') as f:
        for text, label in train_data:
            f.write(f'__label__{label} {text}\n')

In [312]:
txts_to_datset("./professional_lines.txt", "./not_professional_lines.txt", "./test.txt")
txts_to_datset("../tests/professional_lines.txt", "../tests/non_professional_lines.txt", "./train.txt")

40401it [00:00, 152995.58it/s]
111900it [00:00, 168554.35it/s]
419it [00:00, 140010.63it/s]
657it [00:00, 123450.31it/s]


In [313]:
model = fasttext.train_supervised('./train.txt', epoch=100, lr=0.01)

Read 0M words
Number of words:  966
Number of labels: 2
Progress: 100.0% words/sec/thread:  521241 lr:  0.000000 avg.loss:  0.157260 ETA:   0h 0m 0s


In [318]:
model.predict(preprocess("cllen ting on 2022-01-22 $98.17"))

(('__label__not_professional',), array([0.99077946]))

In [316]:
model.test("./train.txt"), model.test("./test.txt")

((838, 0.9952267303102625, 0.9952267303102625),
 (80802, 0.9633301155911982, 0.9633301155911982))